# **Laboratorio 4**

Angélica Ortiz - 202222480 
<br>
María José Amorocho - 202220179

# Importaciones

In [43]:
#importaciones
import numpy as np
from tabulate import tabulate

# Problema 1 - Implementación del Método Simplex Estándar

## Implementación

In [51]:
def simplex_con_iteraciones(A, b, c, imprimir=True, modo='max'):
    """
    Implementa el Método Simplex y retorna los valores finales 
    de todas las variables (originales y de holgura).
    
    Máx  z = c^T x
    s.a. A x <= b
         x >= 0

    Parámetros:
    -----------
    A :  Matriz de coeficientes de las restricciones (m x n)
    b :  Vector de términos independientes (m,1)
    c :  Vector de coeficientes de la función objetivo (n,)
    imprimir : Si es True, se imprime la tabla y detalles en cada iteración
    modo: si es problema de maximización (max) o minimización (min)

    Retorna:
    --------
    resultados : Diccionario con los valores finales de las variables originales y de holgura.
    z_opt : Valor óptimo de la función objetivo.
    """
    # Establecer numero de restricciones y de variables
    m, n = A.shape
    
    # Si es un problema de minimización, convertirlo a maximización
    if modo == 'min':
        c = -c
    
    # Establecer matriz de identidad para variables de holgura
    I = np.eye(m)
    
    # Construccion de la matriz A con las variables de holgura incluidas
    A_ext = np.hstack([A, I]) #matriz de tamaño m x (n+m)
    
    #Extender el vector c para incluir las variables de holgura
    c_ext = np.concatenate([c, np.zeros(m)])
    
    # Construir la tabla: dimensiones (m+1) x (n+m+1)
    # (m+1) filas > m restricciones + 1 fila de función objetivo
    # (n+m+1) columnas > n+m variables (originales y de holgura) + 1columna para terminos independientes (b)
    matriz_simplex = np.zeros((m+1, n+m+1))
    
    # Llenar las filas de restricciones
    for i in range(m):
        matriz_simplex[i, :-1] = A_ext[i]
        matriz_simplex[i, -1]  = b[i]
    
    # Fila de la función objetivo (utilizando -c para la maximización)
    matriz_simplex[-1, :-1] = -c_ext
    
    # iniciar las iteraciones
    iteracion = 0
    while True:
        if imprimir:
            print(f"Iteración {iteracion}:")
            print(tabulate(matriz_simplex, floatfmt=".3f", tablefmt="grid"))
            print("-" * 50)
        
        # 1. Buscar la columna pivote (más negativa en la fila de la función objetivo)
        fila_objetivo = matriz_simplex[-1, :-1]
        col_in = np.argmin(fila_objetivo)
        valor_min = fila_objetivo[col_in]
        
        # si todos los valores en la fila de la función objetivo son positivos se ha encontrado el optimo
        if valor_min >= 0:
            if imprimir:
                print("Óptimo alcanzado.")
            break  # Solución óptima alcanzada
        
        # 2. Seleccionar la fila pivote usando el cociente mínimo
        columna_pivote = matriz_simplex[:-1, col_in]
        b_column = matriz_simplex[:-1, -1]
        
        # calcular las divisiones o razones
        razones = []
        for i in range(m):
            if columna_pivote[i] > 1e-15: # manejar divisiones por cero o muy pequeñas
                razones.append(b_column[i] / columna_pivote[i])
            else:
                razones.append(np.inf)
                
         # La fila con la razón mínima es la que sale de la base
        fila_out = np.argmin(razones)
        if razones[fila_out] == np.inf:
            # Problema no acotado > cuando todas las razones calculadas dan infinito
            raise ValueError("El problema es no acotado.")
        
        if imprimir:
            print(f"Columna que entra: {col_in+1}")
            print(f"Fila que sale: {fila_out+1}")
            print(f"Valor pivote: {matriz_simplex[fila_out, col_in]}")
        
        # 3. normalizar la fila pivote para que el elemento pivote sea 1
        pivote = matriz_simplex[fila_out, col_in]
        matriz_simplex[fila_out, :] /= pivote
        
        # Copnvetir en ceros en el resto de la columna pivote
        for i in range(m+1):
            if i != fila_out:
                factor = matriz_simplex[i, col_in]
                matriz_simplex[i, :] -= factor * matriz_simplex[fila_out, :]
        
        iteracion += 1
        
    # Valor óptimo obtenido en la tabla
    z_opt = matriz_simplex[-1, -1]
    # Para problemas de minimización, revertir el signo del óptimo
    if modo == 'min':
        z_opt = -z_opt
    
    return matriz_simplex, z_opt

def extraer_resultados_simplex(matriz_simplex, A):
    m, n = A.shape
    
    # Extraer la solución óptima > se revisan todas las columnas (originales + holgura)
    x_opt = np.zeros(n + m)
    for col_index in range(n + m):
        # Extraer la columna correspondiente a las filas de restricciones
        col = matriz_simplex[:m, col_index]
        # verificar si es columna canónica (un 1 y el resto 0's)
        if np.count_nonzero(col) == 1 and np.isclose(np.max(col), 1.0):
            row_index = np.argmax(col)
            x_opt[col_index] = matriz_simplex[row_index, -1]
    
    # diccionario para diferenciar variables originales y de holgura
    resultados = {}
    for i in range(n):
        resultados[f'x{i+1}'] = x_opt[i]
    for i in range(m):
        resultados[f's{i+1}'] = x_opt[n + i]
    
    return resultados
    
    

## Primera ejecución

In [55]:
# Ejemplo:
# Máx z = 3x1 + 2x2 + 5x3
# Restricciones
#   x1  +  x2  +  x3 <= 100
#   2x1 +  x2  +  x3 <= 150
#   x1  +  4x2 + 2x3 <= 80
# x1, x2, x3 >= 0

A = np.array([
    [1, 1, 1],
    [2, 1, 1],
    [1, 4, 2]
], dtype=float)

b = np.array([100, 150, 80], dtype=float)
c = np.array([3, 2, 5], dtype=float)

matriz_simplex, valor_objetivo = simplex_con_iteraciones(A, b, c, imprimir=True, modo='max')
solucion = extraer_resultados_simplex(matriz_simplex, A)

print("\nValor final de las variables:")
for i in solucion:
    print(f'> {i}: {round(solucion[i], 3)}')
print("Valor óptimo de la función objetivo:", round(valor_objetivo, 3))

Iteración 0:
+--------+--------+--------+--------+--------+--------+---------+
|  1.000 |  1.000 |  1.000 |  1.000 |  0.000 |  0.000 | 100.000 |
+--------+--------+--------+--------+--------+--------+---------+
|  2.000 |  1.000 |  1.000 |  0.000 |  1.000 |  0.000 | 150.000 |
+--------+--------+--------+--------+--------+--------+---------+
|  1.000 |  4.000 |  2.000 |  0.000 |  0.000 |  1.000 |  80.000 |
+--------+--------+--------+--------+--------+--------+---------+
| -3.000 | -2.000 | -5.000 | -0.000 | -0.000 | -0.000 |   0.000 |
+--------+--------+--------+--------+--------+--------+---------+
--------------------------------------------------
Columna que entra: 3
Fila que sale: 3
Valor pivote: 2.0
Iteración 1:
+--------+--------+-------+-------+-------+--------+---------+
|  0.500 | -1.000 | 0.000 | 1.000 | 0.000 | -0.500 |  60.000 |
+--------+--------+-------+-------+-------+--------+---------+
|  1.500 | -1.000 | 0.000 | 0.000 | 1.000 | -0.500 | 110.000 |
+--------+--------+--

## Segunda ejecución

En esta ejecución, los coeficientes de la función objetivo son modificados con el fin de, posteriormente, hacer un análisis de sensibilidad

In [56]:
# Ejemplo:
# Máx z = 2.5x1 + 1.5x2 + 4.5x3
# Restricciones
#   x1  +  x2  +  x3 <= 100
#   2x1 +  x2  +  x3 <= 150
#   x1  +  4x2 + 2x3 <= 80
# x1, x2, x3 >= 0

A = np.array([
    [1, 1, 1],
    [2, 1, 1],
    [1, 4, 2]
], dtype=float)

b = np.array([100, 150, 80], dtype=float)
c = np.array([2.5, 1.5, 4.5], dtype=float)

matriz_simplex, valor_objetivo = simplex_con_iteraciones(A, b, c, imprimir=False, modo='max')
solucion = extraer_resultados_simplex(matriz_simplex, A)
print("\nValor final de las variables:")
for i in solucion:
    print(f'> {i}: {round(solucion[i], 3)}')
print("Valor óptimo de la función objetivo:", round(valor_objetivo, 3))


Valor final de las variables:
> x1: 73.333
> x2: 0.0
> x3: 3.333
> s1: 23.333
> s2: 0.0
> s3: 0.0
Valor óptimo de la función objetivo: 198.333


## Tercera ejecución

En esta ejecución los términos independientes que afectan la solución son modificados

In [57]:
# Ejemplo:
# Máx z = 3x1 + 2x2 + 5x3
# Restricciones
#   x1  +  x2  +  x3 <= 98
#   2x1 +  x2  +  x3 <= 148
#   x1  +  4x2 + 2x3 <= 78
# x1, x2, x3 >= 0

A = np.array([
    [1, 1, 1],
    [2, 1, 1],
    [1, 4, 2]
], dtype=float)

b = np.array([98, 148, 78], dtype=float)
c = np.array([3, 2, 5], dtype=float)

matriz_simplex, valor_objetivo = simplex_con_iteraciones(A, b, c, imprimir=False, modo='max')
solucion = extraer_resultados_simplex(matriz_simplex, A)
print("\nValor final de las variables:")
for i in solucion:
    print(f'> {i}: {round(solucion[i], 3)}')
print("Valor óptimo de la función objetivo:", round(valor_objetivo, 3))


Valor final de las variables:
> x1: 72.667
> x2: 0.0
> x3: 2.667
> s1: 22.667
> s2: 0.0
> s3: 0.0
Valor óptimo de la función objetivo: 231.333


# Problema 2 - Implementación del Método Simplex Dual Phase

# Problema 3 - Comparación de Rendimiento con GLPK/Pyomo

# Problema 4 - Análisis de Sensibilidad en Programación Lineal

## Análisis de Sensibilidad para Coeficientes de la Función Objetivo

## Análisis de Sensibilidad para Términos Independientes